In [1]:
import torch #The torch package contains data structures for multi-dimensional tensors and mathematical operations over these are defined.
import torchvision #The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms


# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189) 

In [2]:
# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
"""
If we set x, w, b 's required_grad = False, there will be an error:
RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn
"""
y.backward()

# Print out the gradients.
"""
type(x): torch.Tensor
type(x.grad): torch.Tensor
"""
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 

tensor(2.)
tensor(1.)
tensor(1.)


In [3]:
# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2) #type: torch.nn.modules.linear.Linear
print ('w: ', linear.weight) #type: torch.nn.parameter.Parameter
print ('b: ', linear.bias) #type: torch.nn.parameter.Parameter

# Build loss function and optimizer.
criterion = nn.MSELoss() #type: torch.nn.modules.loss.MSELoss
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01) #torch.optim.sgd.SGD

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.\
"""
Computes the gradient of current tensor w.r.t. graph leaves.
This function accumulates gradients in the leaves - you might need to
    zero them before calling it!
"""
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

if False:
    # 1-step gradient descent.
    """Performs a single optimization step."""
    #adapted from https://pytorch.org/docs/stable/optim.html
    #loss.backward() possibly redundant
    def closure():
        optimizer.zero_grad()
        pred = linear(x)
        loss = criterion(pred, y)
        #loss.backward()
        return loss
    # torch.optim.sgd.SGD.step()'s argument closure: 
    #  A closure that reevaluates the model and returns the loss.
    loss_from_step = optimizer.step(closure=closure)
    print('loss from step', loss_from_step)
else:
    # You can also perform gradient descent at the low level.
    linear.weight.data.sub_(0.01 * linear.weight.grad.data)
    linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
"""Use torch.Tensor.item() to get python number"""
"""(torch.Tensor.data returns a torch.Tensor)"""
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[ 0.2032, -0.0122, -0.0504],
        [-0.5185, -0.0977,  0.5375]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.5232, -0.1430], requires_grad=True)
loss:  1.7934141159057617
dL/dw:  tensor([[-0.0347, -0.5567,  0.1363],
        [ 0.1165, -0.3818,  0.8964]])
dL/db:  tensor([ 0.7697, -0.6838])
loss after 1 step optimization:  1.7700774669647217


In [4]:
"""
Difference between sub and sub_
"""
a = torch.tensor(2)
b = torch.tensor(1)
print('a:',a,'b:',b)
print('a-b using sub', a.data.sub(b))
print('after sub', 'a:', a, 'b:', b)
print('a-b using sub_', a.data.sub_(b))
print('after sub_', 'a:', a, 'b:', b) #sub_ is inplace version

a: tensor(2) b: tensor(1)
a-b using sub tensor(1)
after sub a: tensor(2) b: tensor(1)
a-b using sub_ tensor(1)
after sub_ a: tensor(1) b: tensor(1)


In [5]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
print('y:', y)

# Convert the torch tensor to a numpy array.
z = y.numpy()
print('z:', z)
if np.all(x==z):
    print('x equals to z!')

y: tensor([[1, 2],
        [3, 4]])
z: [[1 2]
 [3 4]]
x equals to z!


In [6]:
# ================================================================== #
#                         4. Input pipline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print ('image size:', image.size())
print ('label:', label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()
print('images size:', images.size())
print('labels:', labels)

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

Files already downloaded and verified
image size: torch.Size([3, 32, 32])
label: 6
images size: torch.Size([64, 3, 32, 32])
labels: tensor([8, 8, 7, 0, 8, 7, 0, 2, 9, 2, 9, 5, 0, 3, 6, 3, 8, 1, 3, 4, 3, 9, 2, 2,
        8, 0, 1, 4, 2, 2, 8, 5, 9, 3, 2, 8, 5, 0, 6, 5, 0, 7, 3, 2, 7, 3, 4, 5,
        2, 3, 4, 2, 5, 6, 3, 0, 1, 1, 1, 8, 7, 1, 7, 8])


In [ ]:
# This cannot be run!
# ================================================================== #
#                5. Input pipline for custom dataset                 #
# ================================================================== #

# You should your build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

In [7]:
# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
# resnet18 architecture here: https://github.com/szagoruyko/functional-zoo/blob/master/resnet-18-export.ipynb
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
# resnet.fc.in_features: 512
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

torch.Size([64, 100])


In [8]:
# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))